In [7]:
s = 'RPITTVWCHNHCTAYMTVRVWYHVLADTVMVSCSRCLTNMNMPGKTAEDTKGWLANYAVTKDDGYLVQSCWYYYEDCFTTKRTKKGTDKNNRMPWNISA'
t = 'NQNMMKLQFTWKFNNGDLMLWCHQHDMRHNFYFHLRINMRISTDMVGICDMLFGVYLPLWLYRPACGRECGTTMWVGVDMASYARYVIHGTIYLCRQPL'

In [8]:
import pandas as pd

In [9]:
blosum62 = pd.read_csv("https://github.com/gdv/qomics/raw/main/BLOSUM62.csv")
blosum62

,C,S,T,P,A,G,N,D,E,Q,H,R,K,M,I,L,V,F,Y,W
0,9,-1,-1,-3,0,-3,-3,-3,-4,-3,-3,-3,-3,-1,-1,-1,-1,-2,-2,-2
1,-1,4,1,-1,1,0,1,0,0,0,-1,-1,0,-1,-2,-2,-2,-2,-2,-3
2,-1,1,4,1,-1,1,0,1,0,0,0,-1,0,-1,-2,-2,-2,-2,-2,-3
3,-3,-1,1,7,-1,-2,-1,-1,-1,-1,-2,-2,-1,-2,-3,-3,-2,-4,-3,-4
4,0,1,-1,-1,4,0,-1,-2,-1,-1,-2,-1,-1,-1,-1,-1,-2,-2,-2,-3
5,-3,0,1,-2,0,6,-2,-1,-2,-2,-2,-2,-2,-3,-4,-4,0,-3,-3,-2
6,-3,1,0,-2,-2,0,6,1,0,0,-1,0,0,-2,-3,-3,-3,-3,-2,-4
7,-3,0,1,-1,-2,-1,1,6,2,0,-1,-2,-1,-3,-3,-4,-3,-3,-3,-4
8,-4,0,0,-1,-1,-2,0,2,5,2,0,0,1,-2,-3,-3,-3,-3,-2,-3
9,-3,0,0,-1,-1,-2,0,0,2,5,0,1,1,0,-3,-2,-2,-3,-1,-2


In [10]:
aa = list(blosum62.columns)

In [11]:
bl = {}
for x in range(20):
    bl[aa[x]] = {}
    for y in aa:
        bl[aa[x]][y] = blosum62.iloc[x][y]

In [12]:
indel_penalty = -4

In [13]:
a = []
for i in range(len(s) + 1):
    a.append([(-999, -999, -999)] * (len(t) + 1))

a[0][0] = (0, -1, -1)
for i in range(1, len(s) + 1):
    a[i][0] = (0, -1, -1)
for j in range(1, len(t) + 1):
    a[0][j] = (0, -1, -1)

In [14]:
import time

In [17]:
(max_value, max_i, max_j) = (0, -1, -1)
for i in range(1, len(s) + 1):
    for j in range(1, len(t) + 1):
        a[i][j] = (0, -1, -1)
        if a[i-1][j-1][0] + bl[s[i-1]][t[j-1]] > a[i][j][0]:
            a[i][j] = (a[i-1][j-1][0] + bl[s[i-1]][t[j-1]], i-1, j-1)
        if a[i-1][j][0] + indel_penalty > a[i][j][0]:
            a[i][j] = (a[i-1][j][0] + indel_penalty, i-1, j)
        if (a[i][j-1][0] + indel_penalty) > a[i][j][0]:
            a[i][j] = (a[i][j-1][0] + indel_penalty, i, j-1)
        if a[i][j][0] > max_value:
            (max_value, max_i, max_j) = (a[i][j][0], a[i][j][1], a[i][j][2])

In [34]:
(i, j) = (max_i, max_j)
(value, prev_i, prev_j) = a[i][j]

columns = []
while (value > 0):
    last_column = []
    if (prev_i == i - 1 and prev_j == j -1):
        last_column = (s[i-1], t[j-1])
    if (prev_i == i - 1 and prev_j == j):
        last_column = (s[i-1], '-') 
    if (prev_i == i  and prev_j == j - 1):
        last_column = ('-', t[j - 1])
    
    columns.append(last_column)
    # After the following instructions, (i,j) will be the coordinates of the cell in the next iteration
    (i, j) = (prev_i, prev_j)
    (value, prev_i, prev_j) = a[i][j]
    
    
aligned_s = ''.join([pair[0] for pair in columns[::-1]])
aligned_t = ''.join([pair[1] for pair in columns[::-1]])
cigar = ''
for (c1, c2) in columns[::-1]:
    if c1 == '-' or c2 == '-':
        cigar += ' '
    elif c1 == c2:
        cigar += 'M'
    else:
        cigar += '*'
        
print(aligned_s)
print(cigar)
print(aligned_t)
print(max_value)

VWCHNH-C-TA-YMTVRVWYHVLADTVMVSCSRCLTNMNM
*MMM*M * ** M***M*******M*M** M* *M*****
LWCHQHDMRHNFYFHLRINMRISTDMVGI-CD-MLFGVYL
50
